In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

Using TensorFlow backend.


In [4]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    ds = pydicom.dcmread(filename)    
    if (ds.BodyPartExamined !='CHEST') or (ds.Modality !='DX') or (ds.PatientPosition not in ['AP','PA']):
        img = None
    else:
        print('Load file {} ...'.format(filename))
        img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size): 
    # todo
    proc_img = (img - np.mean(img)) / np.std(img)
    proc_img = np.resize(proc_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, 'r') as json_file:
        model_file = json_file.read()
        model = keras.models.model_from_json(model_file)
        model.load_weights(weight_path)
    
#     model = keras.models.model_from_json(open(model_path, 'r').read())
#     model.load_weights(weight_path)   
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo
    result = model.predict(img)
    predict = result[0]
    if predict > thresh:
        prediction = 'Pneumonia'
    else:
        prediction='No pneumonia'
    
    return prediction 

In [5]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" #path to saved model
weight_path = "pneumonia_class_my_model.best.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.459 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Pneumonia
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Pneumonia


In [21]:
patients_ages = []
patients_sex = []

for i in test_dicoms:    
    ds = pydicom.dcmread(i)
    patients_ages.append(ds.PatientAge)
    patients_sex.append(ds.PatientSex)

In [23]:
patients_ages

['81', '58', '77', '81', '81', '81']

In [24]:
patients_sex

['M', 'M', 'M', 'M', 'M', 'M']